## For Colab Only

In [1]:
%pip install 'git+https://github.com/facebookresearch/detectron2.git'

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'git+https://github.com/facebookresearch/detectron2.git'"


In [2]:
!pip show detectron2

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
video_path = "/content/drive/MyDrive/sample.mp4"

## For Local System

In [1]:
%reload_ext watermark
%watermark -v -p numpy,pandas,pycocotools,torch,torchvision,detectron2,tensorflow,deepface

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.20.0

numpy      : 1.23.5
pandas     : 2.1.4
pycocotools: 2.0.6
torch      : 2.2.1
torchvision: 0.17.1
detectron2 : 0.6
tensorflow : 2.12.0
deepface   : 0.0.89



In [5]:
import torch
import cv2
import numpy as np
import os
import datetime
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.model_zoo import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import ntpath
import pandas as pd
from PIL import Image
import urllib.request
import matplotlib.pyplot as plt
import itertools
import torchvision

# Load face detection model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.DEVICE = 'cpu'  # Set device to CPU
predictor = DefaultPredictor(cfg)

def detect_faces(frame):
    outputs = predictor(frame)
    instances = outputs["instances"]
    pred_classes = instances.pred_classes if instances.has("pred_classes") else None
    boxes = instances.pred_boxes if instances.has("pred_boxes") else None

    detected_faces = []
    if pred_classes is not None and boxes is not None:
        for class_idx, box in zip(pred_classes, boxes):
            if class_idx == 0:  # 0 corresponds to 'person' class in COCO dataset
                box = box.cpu().numpy().astype(int)  # Convert box tensor to numpy array
                x0, y0, x1, y1 = box[0], box[1], box[2], box[3]
                detected_faces.append(frame[y0:y1, x0:x1])
    return detected_faces

def format_timestamp(timestamp_ms):
    # Convert timestamp (in milliseconds) to a datetime object
    timestamp = datetime.datetime.utcfromtimestamp(timestamp_ms / 1000.0).strftime("%H%M%S")
    return timestamp

def save_faces(detected_faces, output_dir, timestamp_ms, frame_number, image_count):
    # Format the timestamp to HHMMSS format
    timestamp = format_timestamp(timestamp_ms)

    for i, face in enumerate(detected_faces):
        face_name = f"{timestamp}_{frame_number}_{image_count + i}.jpg"
        save_path = os.path.join(output_dir, face_name)
        cv2.imwrite(save_path, face)
        print(f"Saved image: {face_name}")
    return len(detected_faces)

def add_frame_number(frame, frame_number):
    font = cv2.FONT_HERSHEY_SIMPLEX
    position = (10, 30)
    font_scale = 1
    font_color = (0, 255, 0)  # Green color
    line_type = 2

    cv2.putText(frame, f"Frame: {frame_number}", position, font, font_scale, font_color, line_type)

def process_video(video_path, output_dir):
    # Open video file
    cap = cv2.VideoCapture(video_path)

    frame_number = 0
    total_detected_images = 0
    frame_skip = 6  # Skip 8 frames after reading one frame

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get timestamp of the current frame in milliseconds
        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

        # Read only every 7th frame
        if frame_number % 7 == 0:
            # Detect faces in the frame
            detected_faces = detect_faces(frame)

            # Save detected faces
            image_count = total_detected_images
            count = save_faces(detected_faces, output_dir, timestamp_ms, frame_number, image_count)
            total_detected_images += count

        frame_number += 1
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number * frame_skip)

    # Release video capture object
    cap.release()
    cv2.destroyAllWindows()

    return total_detected_images

# Create output directory
output_dir = "detected_faces"
os.makedirs(output_dir, exist_ok=True)

# Prompt user to input video path
video_path = input("Enter the path of the video file: ")

# Process video
total_detected_images = process_video(video_path, output_dir)

print("Faces detected and saved successfully.")
print("Total detected images:", total_detected_images)

Saved image: 000005_21_0.jpg
Saved image: 000005_21_1.jpg
Saved image: 000005_21_2.jpg
Saved image: 000005_21_3.jpg
Saved image: 000005_21_4.jpg
Saved image: 000005_21_5.jpg
Saved image: 000005_21_6.jpg
Saved image: 000005_21_7.jpg
Saved image: 000005_21_8.jpg
Saved image: 000005_21_9.jpg
Saved image: 000005_21_10.jpg
Saved image: 000006_28_11.jpg
Saved image: 000006_28_12.jpg
Saved image: 000006_28_13.jpg
Saved image: 000006_28_14.jpg
Saved image: 000006_28_15.jpg
Saved image: 000006_28_16.jpg
Saved image: 000006_28_17.jpg
Saved image: 000006_28_18.jpg
Saved image: 000006_28_19.jpg
Saved image: 000006_28_20.jpg
Saved image: 000006_28_21.jpg
Saved image: 000006_28_22.jpg
Saved image: 000008_35_23.jpg
Saved image: 000008_35_24.jpg
Saved image: 000008_35_25.jpg
Saved image: 000008_35_26.jpg
Saved image: 000008_35_27.jpg
Saved image: 000008_35_28.jpg
Saved image: 000010_42_29.jpg
Saved image: 000010_42_30.jpg
Saved image: 000010_42_31.jpg
Saved image: 000010_42_32.jpg
Saved image: 000010_

In [9]:
from deepface import DeepFace

img1_path = r"C:\Users\kasth\Documents\Lekhya\Final\detected_faces\000028_119_98.jpg"
img2_path = r"C:\Users\kasth\Documents\Lekhya\Final\testing_images\download.jpeg"

# Perform face verification
dfs = DeepFace.verify(img1_path=img1_path, img2_path=img2_path)

# Print the result
print(dfs)


{'verified': True, 'distance': 0.5092053419760114, 'threshold': 0.68, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 49, 'y': 111, 'w': 562, 'h': 562, 'left_eye': (216, 324), 'right_eye': (422, 325)}, 'img2': {'x': 86, 'y': 42, 'w': 92, 'h': 92, 'left_eye': (114, 74), 'right_eye': (147, 81)}}, 'time': 0.54}


In [11]:
from deepface import DeepFace

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
  "GhostFaceNet",
]

'''#face verification
result = DeepFace.verify(img1_path, 
      img2_path, 
      model_name = models[0]
)'''

#face recognition
dfs = DeepFace.find(img_path = r"C:\Users\kasth\Documents\Lekhya\Final\testing_images\download1.jpeg",
db_path = r"C:\Users\kasth\Documents\Lekhya\Final\detected_faces",
      model_name = models[1]
)

'''#embeddings
embedding_objs = DeepFace.represent(img_path = "img.jpg", 
      model_name = models[2]
)'''

print(dfs)

24-04-07 18:03:30 - Searching C:\Users\kasth\Documents\Lekhya\Final\testing_images\download1.jpeg in 795 length datastore
24-04-07 18:03:30 - find function duration 0.30580878257751465 seconds
[                                            identity  \
0  C:\Users\kasth\Documents\Lekhya\Final\detected...   
1  C:\Users\kasth\Documents\Lekhya\Final\detected...   
2  C:\Users\kasth\Documents\Lekhya\Final\detected...   

                                       hash  target_x  target_y  target_w  \
0  459544757ba7924b66394af500bc6dc27837e824       117        36       161   
1  e20b34f0d34c64d73e84123f4ea71e0dd5bd5bd5        76        28        92   
2  9fb4965b333f82083e90277d9aaa95c8ec8a8ca0        77        37       110   

   target_h  source_x  source_y  source_w  source_h  threshold  distance  
0       161        92        31       112       112        0.4  0.243484  
1        92        92        31       112       112        0.4  0.347233  
2       110        92        31       112      

In [14]:
from deepface import DeepFace

models = [
  "VGG-Face",
  "Facenet",
  "Facenet512",
  "OpenFace",
  "DeepFace",
  "DeepID",
  "ArcFace",
  "Dlib",
  "SFace",
  "GhostFaceNet",
]

img_path = r"C:\Users\kasth\Documents\Lekhya\Final\testing_images\download1.jpeg"
db_path = r"C:\Users\kasth\Documents\Lekhya\Final\detected_faces"

results = {}

for model in models:
    dfs = DeepFace.find(img_path=img_path,
                        db_path=db_path,
                        model_name=model)
    results[model + "_dfs"] = dfs


24-04-07 18:14:52 - Searching C:\Users\kasth\Documents\Lekhya\Final\testing_images\download1.jpeg in 795 length datastore
24-04-07 18:14:53 - find function duration 0.39742350578308105 seconds
24-04-07 18:14:53 - Searching C:\Users\kasth\Documents\Lekhya\Final\testing_images\download1.jpeg in 795 length datastore
24-04-07 18:14:53 - find function duration 0.3028066158294678 seconds
24-04-07 18:14:53 - Searching C:\Users\kasth\Documents\Lekhya\Final\testing_images\download1.jpeg in 795 length datastore
24-04-07 18:14:53 - find function duration 0.33957767486572266 seconds
24-04-07 18:14:54 - Searching C:\Users\kasth\Documents\Lekhya\Final\testing_images\download1.jpeg in 795 length datastore
24-04-07 18:14:54 - find function duration 0.15535688400268555 seconds
24-04-07 18:14:54 - Searching C:\Users\kasth\Documents\Lekhya\Final\testing_images\download1.jpeg in 795 length datastore
24-04-07 18:14:55 - find function duration 1.1762666702270508 seconds
24-04-07 18:14:55 - Searching C:\User

In [16]:
print(results)

{'VGG-Face_dfs': [                                            identity  \
0  C:\Users\kasth\Documents\Lekhya\Final\detected...   
1  C:\Users\kasth\Documents\Lekhya\Final\detected...   
2  C:\Users\kasth\Documents\Lekhya\Final\detected...   
3  C:\Users\kasth\Documents\Lekhya\Final\detected...   
4  C:\Users\kasth\Documents\Lekhya\Final\detected...   

                                       hash  target_x  target_y  target_w  \
0  d6f6134df15d9501993c9c0742019d5dbbd2cf0f        65        19        59   
1  ec4230240dfca85d0b9de511c3b166b308ceba06        41        22        62   
2  e20b34f0d34c64d73e84123f4ea71e0dd5bd5bd5        76        28        92   
3  459544757ba7924b66394af500bc6dc27837e824       117        36       161   
4  9fb4965b333f82083e90277d9aaa95c8ec8a8ca0        77        37       110   

   target_h  source_x  source_y  source_w  source_h  threshold  distance  
0        59        92        31       112       112       0.68  0.424567  
1        62        92        31 

In [23]:
with open('face_recognition_results.txt', 'w') as file:
    for model_name, result in results.items():
        file.write(f"Model: {model_name}\n")
        for face in result:
            file.write(f"Face: {face}\n")
        file.write("\n")

In [18]:
import os
import shutil

output_folder = "Same Images"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

common_images = set()

for model_result in results.values():
    for df in model_result:
        image_path = df['identity'][0]
        common_images.add(image_path)

for image_path in common_images:
    image_name = os.path.basename(image_path)
    output_path = os.path.join(output_folder, image_name)
    shutil.copy(image_path, output_path)

print("Images detected by all models are copied to 'Same Images' folder.")


KeyError: 0